In [43]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Invalid requirement: 'torch torchvision torchaudio cudatoolkit=11.2' (from line 7 of requirements.txt)
Hint: = is not a valid operator. Did you mean == ?
Note: you may need to restart the kernel to use updated packages.


In [44]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import datasets, transforms
from torchvision.datasets import CelebA # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) # Set for testing purposes, please do not change!  

In [45]:
def show_tensor_images(image_tensor, num_images=25, size=(3, 64, 64)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.axis('off')  # Hide axes for better visualization
    plt.show()

In [46]:
def get_generator_block(input_dim, output_dim):
    '''
    Function for returning a block of the generator's neural network
    given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a generator neural network layer, with a linear transformation 
          followed by a batch normalization and then a relu activation
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True)
    )

In [47]:
# build the generator class
class Generator(nn.Module):
    
    def __init__(self, z_dim=100, im_dim=12288, hidden_dim=64):
        super(Generator, self).__init__()
        # build the neural network
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            get_generator_block(hidden_dim * 8, hidden_dim * 16),
            nn.Linear(hidden_dim * 16, im_dim),
            nn.Tanh()  # Use Tanh activation for the last layer to output values in [-1, 1]
        )
    
    def forward(self, noise):
        return self.gen(noise)

In [48]:
def get_noise(batch_size, z_dim, device='cpu'):
    '''
    Function to generate a batch of noise vectors (z) that will be used as input to the generator.
    
    Parameters:
        batch_size: Size of the batch, a scalar
        z_dim: Dimension of the noise vector (z), a scalar
        device: Device (CPU or GPU) where the tensor will be allocated
    
    Returns:
        A tensor of shape (batch_size, z_dim) containing noise vectors sampled from a normal distribution
    '''
    return torch.randn(batch_size, z_dim, device=device)

In [49]:
def get_discriminator_block(input_dim, output_dim):
    '''
    Discriminator Block
    Function for returning a neural network block of the discriminator given input and output dimensions.
    
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
        
    Returns:
        a discriminator neural network layer, with a linear transformation 
        followed by an nn.LeakyReLU activation with negative slope of 0.2
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)
    )

In [50]:
# build the discriminator class
class Discriminator(nn.Module):
    def __init__(self, im_dim=12288, hidden_dim=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, image):
        return self.disc(image)

In [51]:
pip install gdown

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [62]:
# training our model !!

# parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 200
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.0002

# trasnform
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize image to 64x64
    transforms.ToTensor(),         # Convert image to PyTorch tensor
    # You can add more transformations here as needed
])

# load func as dataloader
dataloader = DataLoader(
    CelebA('img_align_celeba', download=False, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)


RuntimeError: Dataset not found or corrupted. You can use download=True to download it